In [1]:
import pandas as pd
import re

data = pd.read_csv('data.tsv', sep = '\t', keep_default_na = False, low_memory=False)

In [2]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer, PorterStemmer

def delSW(text):
  stop_words = stopwords.words('english')
  stop_words.extend([',','.',';',':','(',')',"'",'s'])
  return [word for word in text if word not in stop_words]

def lem(text):
  wnl = WordNetLemmatizer()
  return [wnl.lemmatize(w) for w in text]

def stem(text):
  array = []
  for word in text:
    stemmer = SnowballStemmer('english')
    word = stemmer.stem(word)
    array.append(word)
  return array

nltk_data = data

# nltk_data['tokened'] = nltk_data.abstract.str.lower().apply(word_tokenize).apply(delSW)
# nltk_data['lemmed'] = nltk_data.tokened.apply(lem)
# nltk_data['stemmed'] = nltk_data.tokened.apply(stem)

nltk_data.to_csv('nltk_data_prepped.tsv', sep="\t")

In [7]:
from collections import Counter
Counter(''.join(str(v).replace('"','')
  .replace("'",'')
  .replace('[','')
  .replace(']','')
  .replace(',','') for v in nltk_data.lemmed).split()).most_common(10)

[('firm', 59073),
 ('study', 35965),
 ('s', 35157),
 ('model', 34665),
 ('effect', 33078),
 ('result', 29513),
 ('research', 25971),
 ('market', 25653),
 ('find', 24762),
 ('performance', 24605)]

In [3]:
gensim_data = data
gensim_data['abst'] = gensim_data.abstract.map(lambda x: re.sub(r'[^\w\s]', '', x))
gensim_data['abst'] = gensim_data.abst.map(lambda x: x.lower())
gensim_data.abst.head()
gensim_data.to_csv('gensim_data_prepped.tsv', sep="\t")

In [100]:
import gensim
import spacy
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words_gensim = stopwords.words('english')
stop_words_gensim.extend(['from', 'subject', 're', 'edu', 'use', 'research', 'study'])

def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(text):
  return [[word for word in simple_preprocess(str(doc)) if word not in stop_words_gensim] for doc in text]

nlp = spacy.load("en_core_web_sm")
abstracts_isolated = gensim_data.abst.apply(lambda x: " ".join([y.lemma_ for y in nlp(x)]))

In [10]:
abstracts_isolated = pd.read_csv('lemmed.tsv', sep = '\t').drop('Unnamed: 0', axis = 'columns')

In [103]:
abstracts_isolated_edited = abstracts_isolated.values.tolist()
abstracts_words_isolated = list(sent_to_words(abstracts_isolated_edited))
abstracts_words_isolated = remove_stopwords(abstracts_isolated_edited)

In [101]:
abstracts_isolated.to_csv('lemmed.tsv', sep = '\t')

In [104]:
import gensim.corpora as corpora

id2word = corpora.Dictionary(abstracts_words_isolated)
corpus = [id2word.doc2bow(text) for text in abstracts_words_isolated]
lda_model = gensim.models.LdaMulticore(corpus = corpus, id2word = id2word, num_topics = 10)

In [ ]:
from pyLDAvis import gensim_models as gensimvis
from pyLDAvis import enable_notebook, save_html
enable_notebook()

model = gensimvis.prepare(lda_model, corpus, id2word)
save_html(model, 'viz.html')
model

In [183]:
from countrygroups import EUROPEAN_UNION

countries = EUROPEAN_UNION.names
countries.extend(['England','Wales','Scotland'])
type(countries)
schools = data.addresses
schools = schools.str.extractall(r'(?<=\] )(.*?)(?=\;|$)').melt().drop('variable', axis = 1)
schools['university'] = schools.value.str.extract(r'^(.+?),')
schools['country'] = schools.value.str.extract(r'^.*\, (.*)$')
schools = schools[schools.country.isin(countries)].drop_duplicates(subset = 'university')


In [30]:
ft_list = pd.read_csv('list_of_schools.csv', sep = ';').drop('#', axis = 1)

In [114]:
from fuzzywuzzy import fuzz

for school in schools:


<built-in method capitalize of str object at 0x104b2bf70>
